Kudos to: 

* https://www.kaggle.com/code/paultimothymooney/kaggle-support-assistant-mistral-7b-rag/notebook
* https://huggingface.co/docs/transformers/index
* https://python.langchain.com/docs/get_started/introduction
* https://www.kaggle.com/code/philculliton/talking-papers-with-mistral-7b/
* https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

In [1]:
!pip install -qU langchain accelerate bitsandbytes transformers chromadb sentence-transformers faiss-gpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, b

In [2]:
import os
import numpy as np
import pandas as pd
import transformers
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, AutoTokenizer,
)

from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from torch import cuda, bfloat16
import torch
from time import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
class CFG:
    model_path = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
    temperature = 0.7
    repetition_penalty = 1.1
    max_new_tokens = 2000
    model_name = "sentence-transformers/all-mpnet-base-v2"
    rag_data = "/kaggle/input/eu-ai-act-full-text/AIAct_final_four-column21012024.pdf"

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

# Functions

In [5]:
def test_model(tokenizer, pipeline, prompt_to_test):
    
    time_1 = time()
    sequences = pipeline(
        prompt_to_test, do_sample=True,
        top_k=10, num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        

In [6]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

# Model

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    CFG.model_path, quantization_config = bnb_config, do_sample=True,
)
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    temperature= CFG.temperature,    
    task="text-generation",
    repetition_penalty= CFG.repetition_penalty,
    return_full_text=True,
    max_new_tokens= CFG.max_new_tokens,    
)

In [9]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

embeddings = HuggingFaceEmbeddings(model_name= CFG.model_name, model_kwargs={"device": "cuda"})


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# The data

In [10]:
loader = PyPDFLoader(CFG.rag_data)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [11]:
print(f'We have created {len(all_splits)} chunks from {len(documents)} pages')

We have created 4925 chunks from 892 pages


# RAG setup

In [12]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,  chain_type="stuff",  retriever=retriever, verbose=True
)

Batches:   0%|          | 0/154 [00:00<?, ?it/s]

# Compare performance

In [13]:
query = "What is general purpose AI?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 53.33 sec.
Result: What is general purpose AI?
A: a computer program that can think and act like a human being
B a computer program that can process large amounts of data quickly
C a computer program that can recognize speech or images
D a computer program that can perform complex calculations
E all of the above

I'd like to know what general purpose AI means exactly, as I see many people claiming it's coming soon. If it means that machines will be able to think and act like humans, then I'm not sure how much closer we are to this than we were 50 years ago.
User 5: > A a computer program that can think and act like a human being

No, a general purpose AI system would need to be able to do everything a human can do. That includes things like understanding language, recognizing objects, making decisions based on logic and reasoning, and even dreaming.

> B a computer program that can process large amounts of data quickly

This is an important part of AI systems, but not t

In [14]:
test_rag(qa, query)

Query: What is general purpose AI?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 7.293 sec.

Result:   A general purpose AI system is an AI system that is trained on broad data at scale, is designed for generality of output, and can be adapted to a wide range of distinctive tasks. It is meant to serve a variety of purposes, both for direct use as well as for integration into other AI systems. General purpose AI systems may be used themselves or be components of other high-risk AI systems.


In [15]:
query = "What does the EU AI Act say about open source models?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 8.163 sec.
Result: What does the EU AI Act say about open source models?

The EU AI Act does not explicitly address open source models.

However, it does provide for transparency and accountability in the development and use of AI systems. This includes requirements for documentation and explanation of how AI systems make decisions, as well as the right to data portability and the ability to challenge decisions made by AI systems.

Open source models may be more transparent and explainable than proprietary models, and could potentially meet these requirements. However, it is important to note that open source models may also raise privacy concerns if they are used to process personal data without appropriate safeguards in place.


In [16]:
test_rag(qa, query)

Query: What does the EU AI Act say about open source models?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 6.444 sec.

Result:   The EU AI Act states that open source AI models are not subject to certain obligations if they are made available under a free and open license. However, providers of general purpose AI models must have a policy in place to respect Union law on copyright and related rights. Additionally, users are allowed to use open source software and data, including models, through free and open-source licenses.


In [17]:
query = "What does the AI Act say about copyright?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 4.595 sec.
Result: What does the AI Act say about copyright?
A: Question: Let i be x(2). Let w = 68 - 34. Let x(h) = h**3 - 3*h**2 + h - 1. Suppose -i*r = 5*r + w. Is r equal to 9?
Answer: True


In [18]:
test_rag(qa, query)

Query: What does the AI Act say about copyright?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 8.973 sec.

Result:  

The AI Act says that AI systems released under free and open source licenses are not subject to the obligations laid out in the AI Act, unless they are placed on the market or put into service as high-risk AI systems or an AI system that falls under Title II and IV. Additionally, the AI Act states that providers of general-purpose AI models should put in place a policy to respect Union copyright law, in particular to identify and respect the reservations of rights expressed by rightholders pursuant to Article 4(3) of Directive G.


In [19]:
query = "What is the AI Act interpretation of systemic risk?"
test_model(tokenizer, text_generation_pipeline, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 8.042 sec.
Result: What is the AI Act interpretation of systemic risk?
Answer: The AI Act does not explicitly define systemic risk. However, it does provide a framework for regulating the development and use of high-risk AI systems, which could potentially have systemic impacts if they were to fail or be misused. The Act also includes provisions for establishing an independent regulatory authority to oversee the development and use of AI, as well as requirements for transparency, accountability, and safety in AI systems. These measures are intended to help mitigate the risks associated with high-risk AI and ensure that the technology is used in ways that benefit society as a whole.


In [20]:
test_rag(qa, query)

Query: What is the AI Act interpretation of systemic risk?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 3.867 sec.

Result:   As per the AI Act, systemic risk refers to a significant event, occurring within a short period, which has the potential to cause widespread harm or damage to the financial stability of the European Union or to individual Member States.
